In [47]:
dataverse = "SyntheticDb"
dataset_name = "Locations"
dbHost = "http://localhost:19002"
adm_path = "/opt/asterixdb/datasets/adm/synthetic.adm"
adm_path_local = "./data/adm/synthetic.adm"
insert_count = 5_000_000
query_count = 10_000

In [52]:
import requests
create_database_script = f"""
DROP DATAVERSE {dataverse} IF EXISTS;
CREATE DATAVERSE {dataverse};
USE {dataverse};

CREATE TYPE LocationType AS {{
    id: bigint,
    location: point,
    description: string
}};

CREATE DATASET {dataset_name}(LocationType) PRIMARY KEY id;
CREATE INDEX {dataset_name}Index on {dataset_name}(location) TYPE RTREE ENFORCED;
"""
response = requests.post(f"{dbHost}/query/service", data={'statement': create_database_script})
print(response.text)

{
	"requestID": "b7faab61-641b-417a-9522-8556452d9efe",
	"plans":{},
	"status": "success",
	"metrics": {
		"elapsedTime": "342.629917ms",
		"executionTime": "339.271959ms",
		"compileTime": "0ns",
		"queueWaitTime": "0ns",
		"resultCount": 0,
		"resultSize": 0,
		"processedObjects": 0,
		"bufferCacheHitRatio": "0.00%",
		"bufferCachePageReadCount": 0
	}
}



## Generate Dataset

In [59]:
%reload_ext sql

In [ ]:
import os
from sqlalchemy.engine import create_engine

%load_ext sql
conn = create_engine('duckdb:///data/local.duckdb')
json_path_local = adm_path_local.replace("adm", "json")
if not os.path.exists(os.path.dirname(json_path_local)):
    os.makedirs(os.path.dirname(json_path_local), exist_ok=True)
%sql conn --alias duckdb
%config SqlMagic.named_parameters="enabled" 


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [66]:
%%sql
INSTALL psql FROM community;
INSTALL spatial;
LOAD spatial;
LOAD psql;
CREATE OR REPLACE TABLE points AS
WITH generated_points AS (
    FROM ST_GeneratePoints({min_x: -700, min_y:-700, max_x:700, max_y:700}::BOX_2D, {{insert_count}}, 42)
    SELECT point
)
FROM generated_points
SELECT
    id: ROW_NUMBER() OVER (ORDER BY point),
    location: 'point("' || point.ST_X() || ',' || point.ST_Y() || '")',
    description: 'VXNWtdYabdVXNWtdYabdVXNWtdYabdVXNWtdYabdVXNWtdYabd'
ORDER BY random()

Running query in 'duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Count


In [67]:
%%sql
COPY points TO '{{json_path_local}}' (FORMAT JSON);

Running query in 'duckdb'

Count


In [68]:
with open(adm_path_local, "w") as f:
    for row in open(json_path_local, "r").readlines():
        new_row = row.replace('location":"point(\\"', 'location":point("').replace('\\")","description"', '"),"description"')
        f.write(new_row)

In [69]:
import requests

load_command = f"""
LOAD DATASET {dataset_name} USING localfs
    (("path"="127.0.0.1:///{adm_path}"),("format"="adm"));
"""
response = requests.post(f"{dbHost}/query/service", data={'statement': load_command, 'dataverse': dataverse})
print(response.text)

{
	"requestID": "fa9091f4-237d-4a29-8c6c-0698cff631e3",
	"plans":{},
	"status": "success",
	"metrics": {
		"elapsedTime": "45.211051979s",
		"executionTime": "45.204550437s",
		"compileTime": "0ns",
		"queueWaitTime": "0ns",
		"resultCount": 0,
		"resultSize": 0,
		"processedObjects": 0,
		"bufferCacheHitRatio": "0.00%",
		"bufferCachePageReadCount": 0
	}
}



In [71]:
count_command = f"""
USE {dataverse};
SELECT COUNT(*) FROM {dataset_name};
"""
response = requests.post(f"{dbHost}/query/service", data={'statement': count_command})
print(response.json()["results"])

[{'$1': 5000000}]


In [72]:
import random
from utils import point, rectangle, create_offset_rectangle
query_benchmark_plan_path = f"commands/benchmark_query_plan_{dataverse}_{dataset_name}_{query_count}.sql"

with open(query_benchmark_plan_path, 'w') as f:
    for _ in range(query_count):
        x, y = random.uniform(-700, 700), random.uniform(-700, 700)
        offset_factor = 2
        offset_x, offset_y = random.uniform(1, offset_factor), random.uniform(1, offset_factor)
        start_x, start_y, end_x, end_y = (x-offset_x, y-offset_y, x+offset_x, y+offset_y)
        query = f"SELECT * FROM {dataset_name} WHERE spatial_intersect(location, create_rectangle(create_point({start_x},{start_y}), create_point({end_x},{end_y})));"
        f.write(query + "\n") 

In [73]:
from utils import Logger

logs_filename_parts = [dataset_name, query_count]
logger = Logger(logs_filename_parts, True)
with open(query_benchmark_plan_path, 'r') as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        if not line:
            continue
        response = requests.post(f"{dbHost}/query/service", data={'statement': line, 'dataverse': dataverse,})
        json_response = response.json()
        if response.status_code != 200:
            logger.error({
                "status": json_response.get("status"),
                "http.status": response.status_code,
                "metrics": json_response.get("metrics"),
                "error": json_response.get("errors"),
            })
        else:
            logger.log({
                "status": json_response.get("status"),
                "http.status": response.status_code,
                "result_count": len(json_response.get("results", [])) if json_response.get("results") else None,
                "metrics": json_response.get("metrics"),
            })

2025-04-28 10:24:28 - INFO - {"status": "success", "http.status": 200, "result_count": 27, "metrics": {"elapsedTime": "364.497417ms", "executionTime": "339.884417ms", "compileTime": "197.725292ms", "queueWaitTime": "0ns", "resultCount": 27, "resultSize": 4220, "processedObjects": 54, "bufferCacheHitRatio": "100.00%", "bufferCachePageReadCount": 52}}
2025-04-28 10:24:28 - INFO - {"status": "success", "http.status": 200, "result_count": 25, "metrics": {"elapsedTime": "48.981417ms", "executionTime": "48.072083ms", "compileTime": "21.634ms", "queueWaitTime": "0ns", "resultCount": 25, "resultSize": 3938, "processedObjects": 50, "bufferCacheHitRatio": "100.00%", "bufferCachePageReadCount": 48}}
2025-04-28 10:24:28 - INFO - {"status": "success", "http.status": 200, "result_count": 34, "metrics": {"elapsedTime": "48.293833ms", "executionTime": "47.381584ms", "compileTime": "20.694208ms", "queueWaitTime": "0ns", "resultCount": 34, "resultSize": 5307, "processedObjects": 68, "bufferCacheHitRatio